In [57]:
import pandas as pd
from sqlalchemy import create_engine
import csv

'''For educational purpose only. 11.csv will be read from the current folder of this directory and uploaded 
   to the remote datbase only once (WORM). I have already uploaded the data to the remote database and have
   commented out the code that actually calls the method to load the data from the datafram to remote table
   You will need the following packages:
   pandas
   sqlalchemy
   
   To install sqlalchemy use either of the following commands:
   conda install -c anaconda sqlalchemy 
   or
   pip install SQLAlchemy
   '''

infile = '11.csv'
db = 'datascience'
db_tbl_name = 'raw_data'

'''
    Load a csv file into a dataframe; if csv does not have headers, use the headers arg to create a list of headers; 
    rename unnamed columns to conform to mysql column requirements.
    Our csv file has a header row and hence the column names are read from the same.
'''
def csv_to_df(infile, headers = []):
    if len(headers) == 0:
        df = pd.read_csv(infile)
    else:
        df = pd.read_csv(infile, header = None)
        df.columns = headers
    for r in range(10):
        try:
            df.rename( columns={'Unnamed: {0}'.format(r):'Unnamed{0}'.format(r)},    inplace=True )
        except:
            pass
        df.head()
    return df

'''
Create a mapping of df dtypes to mysql data types (not perfect, but close enough)
'''
def dtype_mapping():
    return {'object' : 'TEXT',
        'int64' : 'INT',
        'float64' : 'FLOAT',
        'datetime64' : 'DATETIME',
        'bool' : 'TINYINT',
        'category' : 'TEXT',
        'timedelta[ns]' : 'TEXT'}
'''
Create a sqlalchemy engine. We are connecting to a remote database hosted for our school operations at nextideacademy.org
'''
def mysql_engine(user = 'datascience', password = '76BW9uULcZcYl9SG', host = 'nextideacademy.org', port = '3306', database = 'datascience'):
    engine = create_engine("mysql://{0}:{1}@{2}:{3}/{4}?charset=utf8".format(user, password, host, port, database))
    return engine

'''
Create a mysql connection from sqlalchemy engine
'''
def mysql_conn(engine):
    conn = engine.raw_connection()
    return conn
'''
Create sql input for table names and types
'''
def gen_tbl_cols_sql(df):
    dmap = dtype_mapping()
    sql = "pi_db_uid INT AUTO_INCREMENT PRIMARY KEY"
    df1 = df.rename(columns = {"" : "nocolname"})
    hdrs = df1.dtypes.index
    hdrs_list = [(hdr, str(df1[hdr].dtype)) for hdr in hdrs]
    for i, hl in enumerate(hdrs_list):
        sql += " ,{0} {1}".format(hl[0], dmap[hl[1]])
    return sql

'''
Create a mysql table from a df
This takes the header information from the csv file and creates the table based on the header row
'''
def create_mysql_tbl_schema(df, conn, db, tbl_name):
    tbl_cols_sql = gen_tbl_cols_sql(df)
    sql = "USE {0}; CREATE TABLE IF NOT EXISTS  {1} ({2})".format(db, tbl_name, tbl_cols_sql)
    print ("Create table SQL:" + sql)
    cur = conn.cursor()
    try:
        print("Creating Table")
        cur.execute(sql)
        print("End of table creation")
        cur.close()
        conn.commit()
    except Exception as inst:
        print(type(inst))    # the exception instance
        print(inst.args)     # arguments stored in .args
        print(inst)  
        pass
    
    

'''
Write df data to newly create mysql table
'''
def df_to_mysql(df, engine, tbl_name):
    df.to_sql(tbl_name, engine, if_exists='replace', index_label='record', index = False)

#Create the DataFrame object 
df = csv_to_df(infile)

#The following line of code creates the table based on the 11.csv file. I will leave it commented.
#create_mysql_tbl_schema(df, mysql_conn(mysql_engine()), db, db_tbl_name)


In [48]:
#Loads 1.8 million records. Don't execute the following line as it's already been done. I promise this works :)
#df_to_mysql(df, mysql_engine(), db_tbl_name)


In [59]:
 
'''
    This block of code is what you will need to load the 1.8 million records from the Remote MySQL database 
    via a cursor to create a DataFrame and wrangle as needed. Since the csv file we are using or any large data
    is not feasible to be carried around in files, we connect to a remote DB for a read once operation.
    Any operations for wrangling (trial and errors) can be done using DataFrame objects. Our instructor said a few million records 
    won't be an issue.
    Once we are satisfied with the wrangling, we will write it to a separate table (another WORM table)
'''
import pandas.io.sql as psql

conn = mysql_conn(mysql_engine())
sql = "SELECT * from " + db_tbl_name
new_df_from_db = df = pd.read_sql(sql, conn)
new_df_from_db.count()

index       1835072
record      1835072
age         1835072
race        1832182
sex         1835072
ms          1473846
hisp        1782570
adjinc      1790803
educ        1484202
pob         1835072
wt          1835072
hhid        1835072
hhnum       1835072
reltrf      1830398
occ          979747
majocc       979747
ind          979526
majind       979526
esr         1484169
urban       1822030
smsast      1822016
inddea      1835072
cause113    1835072
follow      1835072
dayod        160750
hosp         157794
hospd        145174
ssnyn       1835072
vt          1439021
histatus    1257307
hitype      1257307
povpct      1835072
stater      1835072
rcow         981362
tenure      1809625
citizen      464861
health       384129
indalg       348759
smok100           0
agesmk            0
smokstat          0
smokhome          0
curruse           0
everuse           0
dtype: int64

In [55]:
#Printing the good old df counts read from the CSV file to make sure the data matches the remote MySQL DB Table
df.count()

index       1835072
record      1835072
age         1835072
race        1832182
sex         1835072
ms          1473846
hisp        1782570
adjinc      1790803
educ        1484202
pob         1835072
wt          1835072
hhid        1835072
hhnum       1835072
reltrf      1830398
occ          979747
majocc       979747
ind          979526
majind       979526
esr         1484169
urban       1822030
smsast      1822016
inddea      1835072
cause113    1835072
follow      1835072
dayod        160750
hosp         157794
hospd        145174
ssnyn       1835072
vt          1439021
histatus    1257307
hitype      1257307
povpct      1835072
stater      1835072
rcow         981362
tenure      1809625
citizen      464861
health       384129
indalg       348759
smok100           0
agesmk            0
smokstat          0
smokhome          0
curruse           0
everuse           0
dtype: int64